In [101]:
# Task 1.1

# Define the dot product
def dot(list_A,list_B):
    return sum([list_A[i]*list_B[i] for i in range(len(list_A))])

weight = [3,1]*6

def valid_id(string):
    if len(string) != 12:
        print('The string is not a valid identification number.')
        return False
    else:
        string = [int(i) for i in string]
        if dot(weight,string)%10 == 0:
            print('The string is a valid identification number.')
            return True
        else:
            print('The string is not a valid identification number.')
            return False

valid_id('036000291452')

The string is a valid identification number.


True

In [97]:
# Task 1.2

def id_to_barcode(string):
    quiet = '0'*9
    start = '101'
    mid = '01010'
    end = '101'

    encode_table =  {
                        '0': ['0001101','1110010'],
                        '1': ['0011001','1100110'],
                        '2': ['0010011','1101100'],
                        '3': ['0111101','1000010'],
                        '4': ['0100011','1011100'],
                        '5': ['0110001','1001110'],
                        '6': ['0101111','1010000'],
                        '7': ['0111011','1000100'],
                        '8': ['0110111','1001000'],
                        '9': ['0001011','1110100']
                    }

    first_half = [encode_table[string[i]][0] for i in range(6)]
    first_half = ''.join(first_half)
    second_half = [encode_table[string[6+i]][1] for i in range(6)]
    second_half = ''.join(second_half)

    return ''.join([quiet,start,first_half,mid,second_half,end,quiet])

id_to_barcode('036000291452')

'00000000010100011010111101010111100011010001101000110101010110110011101001100110101110010011101101100101000000000'

In [105]:
def valid_start_mid_end(barcode_string):
    quiet = '0'*9
    start = '101'
    mid = '01010'
    end = '101'
    if (barcode_string[:12] == quiet + start) and (barcode_string[54:59] == '01010') and (barcode_string[101:] == end + quiet):
        return True
    return False

def barcode_to_id(barcode_string):
    if valid_start_mid_end(barcode_string):
        first_half = barcode_string[12:54]
        second_half = barcode_string[59:101]
        decode_first_half = {
                            '0001101': '0',
                            '0011001': '1',
                            '0010011': '2',
                            '0111101': '3',
                            '0100011': '4',
                            '0110001': '5',
                            '0101111': '6',
                            '0111011': '7',
                            '0110111': '8',
                            '0001011': '9'
                            }
        decode_second_half = {
                            '1110010':'0',
                            '1100110':'1',
                            '1101100':'2',
                            '1000010':'3',
                            '1011100':'4',
                            '1001110':'5',
                            '1010000':'6',
                            '1000100':'7',
                            '1001000':'8',
                            '1110100':'9'
                            }

        # no of 1s in first part must be odd and later on must be even
        first_odd = all([first_half[7*i:7*i+7].count('1') % 2 ==1 for i in range(6)])
        second_even = all([second_half[7*i:7*i+7].count('1') % 2 == 0 for i in range(6)])

        # Valid barcode and not upside down. 
        if first_odd and second_even:
            decoded_first = [decode_first_half[first_half[7*i:7*i+7]] for i in range(6)]
            decoded_second = [decode_second_half[second_half[7*i:7*i+7]] for i in range(6)]
            return ''.join(decoded_first) + ''.join(decoded_second)
        else:
            # if barcode is upside down and valid:
            if not first_odd  and not second_even:            
                second_half_reversed = ''.join(reversed(first_half))
                first_half_reversed  = ''.join(reversed(second_half))
                decoded_first = [decode_first_half[first_half_reversed[7*i:7*i+7]] for i in range(6)]
                decoded_second = [decode_second_half[second_half_reversed[7*i:7*i+7]] for i in range(6)]
                return ''.join(decoded_first) + ''.join(decoded_second)
    print('Barcode is invalid')
    return False

# barcode_to_id('00000000010100011010111101010111100011010001101000110101010110110011101001100110101110010011101101100101000000000')
terbalik = ''.join(reversed('00000000010100011010111101010111100011010001101000110101010110110011101001100110101110010011101101100101000000000'))
print(terbalik)
print(barcode_to_id(terbalik))

00000000010100110110111001001110101100110010111001101101010101100010110001011000111101010111101011000101000000000
036000291452


In [ ]:
# Task 2

In [45]:
# Task 3

# We will use datetime to make our lives easier, but note that there are some formatting 
# required to use datetime objects.
# timedelta objects can be used to represent the 21 days difference 

from datetime import datetime, timedelta 

date_time_str = '2021-01-05'

def second_dose_date(date):
    date_time_str = f'{date[:4]}-{date[4:6]}-{date[6:]}'
    date_first_vaccine = datetime.strptime(date_time_str,'%Y-%m-%d')
    days_to_vaccine = timedelta(days=21)
    date_second_vaccine = date_first_vaccine + days_to_vaccine
    # Note that we're using zfill() string method to pad zeroes to match the format required
    # another way is to modify the month and day string individually using conditionals
    return f'{str(date_second_vaccine.year)}{str(date_second_vaccine.month).zfill(2)}{str(date_second_vaccine.day).zfill(2)}'

print(second_dose_date('20210105'))
print(second_dose_date('20210212'))
print(second_dose_date('20210919'))

20210126
20210305
20211010


In [46]:
# Task 3.2

from pymongo import MongoClient

try:
    client = MongoClient('localhost', 27017)
    print("Connected successfully!!!")
except:
    print("Could not connect to MongoDB")

db = client['community_centre']

coll = db['management_committee']

# dropping a collection with pymongo
if list(coll.find({})) != []:
    print('Collection is not empty. Dropping collection first.')
    coll.drop()

with open('VACCINATION.TXT') as f:
    reader = f.readlines()
    reader = [ i.strip().split(',') for i in reader]

# the idea is to use insert_many() method of pymongo Collection object
# so first we need to convert the imported list into json-like objects

def list_to_dict(arr):
    member = {}

    # Assume the first two attributes always exist
    member['_id'] = arr[0]
    member['name'] = arr[1]
    
    # handling the dates and remarks field
    i = 2
    
    while i < len(arr):
        # if the entry in the row is an alphabet, it's straight away must be a remark
        if arr[i].isalpha():
            member['remarks'] = arr[i]
        else:
            # check if we have inserted a date as an attribute in the dictionary object
            if 'date_first_dose' in member.keys():
                member['date_second_dose'] = arr[i]
            else:
                member['date_first_dose'] = arr[i]
        i += 1
    return member

list_json = [ list_to_dict(member) for member in reader]

coll.insert_many(list_json)

Connected successfully!!!
Collection is not empty. Dropping collection first.


In [47]:
# Task 3.3

member_id = input('Please input member ID (integer) :')

while not member_id.isdigit():
        if member_id == 'stop':
                break
        member_id = input('Please input a valid member ID (integer) :')

myquery = {'_id': member_id}

member_list = list(coll.find(myquery))

# This function is an afterthought as I want to make the code more modular
def certificate_string(name,date_first_dose,date_second_dose):
        string = [      'VACCINATION CERTIFATE\n',
                        '\n',
                        f'Name : {name}\n',
                        'Vaccine type: CoviDie\n', 
                        f'Date of first dose: {date_first_dose}\n', 
                        f'Date of second dose: {date_second_dose}'
                ]
        return string

if member_list == []:
        print('Member ID is not found in the database.')
else:
        member = member_list[0]
        fields = member.keys()
        if not ('date_first_dose' in fields):
                print('Member is unvaccinated. Please take the first dose as soon as possible.')
        elif 'date_second_dose' in fields:
                with open('certificate.txt','w') as f:
                        name = member['name']
                        date_first_dose = member['date_first_dose']
                        date_second_dose = member['date_second_dose']
                        f.writelines(certificate_string(name,date_first_dose,date_second_dose))
        else:   
                date_first_dose = member['date_first_dose']
                print(f'Member only has first dose. The earliest date to take second dose is {second_dose_date(date_first_dose)}')



dict_keys(['_id', 'name', 'date_first_dose'])
Member only has first dose. The earliest date to take second dose is 20210421


In [44]:
# Task 4.1 

# See folder for the sql file

False

In [2]:
# Task 4.2

import sqlite3

import csv

connection = sqlite3.connect('bento_company.db')

cursor = connection.cursor()

with open('Task4.sql') as f:
    script = f.read()

cursor.executescript(script)

# Insertion in Kiosk table
with open('KIOSK.txt') as f:
    reader = csv.reader(f)
    reader = list(reader)

myinsert = 'INSERT INTO Kiosk(KioskID,Location,Rating) VALUES (?,?,?)'

for row in reader:
    myvals = (row[0],row[1],row[2])
    cursor.execute(myinsert,myvals)


# Insertion in BENTOBOX table
with open('BENTOBOX.txt') as f:
    reader = csv.reader(f)
    reader = list(reader)

myinsert = 'INSERT INTO BentoBox(BentoName,ProductionCost,ContainEgg,ContainNut,ContainSeafood) VALUES (?,?,?,?,?)'

for row in reader:
    myvals = (row[0],row[1],row[2],row[3],row[4])
    cursor.execute(myinsert,myvals)

# INsertion into KioskBento Table
#Note that we can use LEFT OUTER JOIN to get the cartesian product first
#Alternatively, you can do two iterations to insert as well
myquery = 'SELECT * FROM Kiosk LEFT OUTER JOIN BentoBox ON kiosk.KioskID'

queried_list = list(cursor.execute(myquery))

myinsert = 'INSERT INTO KioskBento(KioskID,BentoName,SellPrice) VALUES (?,?,?)'

price_delta = [2.6,2.9,2.4,3.1]

for row in queried_list:
    #Note that KioskID starts from index 1. As such, index used to access elements
    #in price_delta has to be adjusted
    myvals = (row[0],row[3],float(row[4])+price_delta[int(row[0])-1])
    cursor.execute(myinsert,myvals)

connection.commit()

In [12]:
# Task 4.3

from flask import Flask, render_template, request

import sqlite3

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/bento',methods=['GET'])
def form():
    #Note that we're using request.values() here instead of request.form.get()
    #Alternatively, we can use request.args.get() for GET (query string) value
    location = request.values.get('location')
    egg = 0 if request.values.get('egg') == None else 1
    nut = 0 if request.values.get('nut') == None else 1
    seafood = 0 if request.values.get('seafood') == None else 1
    connection = sqlite3.connect('bento_company.db')
    cursor = connection.cursor()
   
    def condition_allergy(egg, nut, seafood):
        allergic_to = []
        allergents = [egg,nut,seafood]
        allergents_string = ['ContainEgg','ContainNut','ContainSeafood']
        for i in range(len(allergents)):
            if allergents[i] != 0:
                allergic_to.append(allergents_string[i])
        if allergic_to == []:
            return ''
        else:
            allergic_string = [f'{i} = 0' for i in allergic_to]
            return ' WHERE ' + ' AND '.join(allergic_string)

    # The approach I'm going for is through modifying query conditions in the sql statement and done in 2 stages
    # 1st. Get all the bentos that fit the requirement
    myquery = f'SELECT BentoName FROM BentoBox{condition_allergy(egg,nut,seafood)}'
    
    bento_names = []
    bento_names =  list(cursor.execute(myquery))
    bento_names = [bento[0] for bento in bento_names]
    
    # Next, we will get the allowed bentos in the location specified
    products = []

    for bento in bento_names:
        
        #We will use Inner Join so that we can get Bentoname and SellPrice from same table
        my_second_query = f"SELECT BentoName, SellPrice FROM Kiosk INNER JOIN KioskBento on Kiosk.KioskID = KioskBento.KioskID WHERE Location = '{location}' AND BentoName = '{bento}'" 
        bento = list(cursor.execute(my_second_query))

        #We access the 0th element because bento is a list of single tuple
        products.append(bento[0])

    return render_template('filtered.html',products=products,location = location)

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Jan/2022 10:17:48] "GET /bento?location=Woodlands&egg=ContainEgg&nut=ContainNut&seafood=ContainSeafood HTTP/1.1" 200 -


Woodlands
Woodlands BentoB
SELECT BentoName, SellPrice FROM Kiosk INNER JOIN KioskBento on Kiosk.KioskID = KioskBento.KioskID WHERE Location = 'Woodlands' AND BentoName = 'BentoB'


127.0.0.1 - - [07/Jan/2022 10:17:52] "GET /bento?location=Woodlands&egg=ContainEgg&seafood=ContainSeafood HTTP/1.1" 200 -


In [2]:
def condition_allergy(egg, nut, seafood):
    allergic_to = []
    allergents = [egg,nut,seafood]
    allergents_string = ['ContainEgg','ContainNut','ContainSeafood']
    for i in range(len(allergents)):
        if allergents[i] != 0:
            allergic_to.append(allergents_string[i])
    if allergic_to == []:
        return ''
    else:
        allergic_string = [f'{i} = 0' for i in allergic_to]
        return ' WHERE ' + ' AND '.join(allergic_string)

myvals = (1,1,1)
condition_allergy(1,0,1)

' WHERE ContainEgg = 0 AND ContainSeafood = 0'

In [15]:
' AND '.join(['x'])

'x'